In [ ]:
import json
import os
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from filter_optimization.filter_optimization_task import extract_biomass_data
from research.weight_estimation.keypoint_utils.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point

from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils

from scipy import stats
import statsmodels.api as sm

plt.rcParams['font.size'] = 18

In [ ]:
pen5 = pd.read_csv('blom_vikane_singleweights.csv')

In [ ]:
pen5.head()

In [ ]:
def get_length_from_weight(weight):
    return 

In [ ]:
queryCache = {}

In [ ]:
# pen_id = 95
# df_start_date = '2020-07-21'
# df_end_date = '2020-07-24'
pen_id = 60
df_start_date = '2020-08-24'
df_end_date = '2020-08-26'

if pen_id in queryCache and df_start_date in queryCache[pen_id] and df_end_date in queryCache[pen_id][df_start_date]:
    df = queryCache[pen_id][df_start_date][df_end_date]
else:
    df = extract_biomass_data(pen_id, df_start_date, df_end_date, 0.95)
    df.date = pd.to_datetime(df.date)

    depths = []
    lengths = []
    for idx, row in df.iterrows():
        ann, cm = row.annotation, row.camera_metadata
        wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
        depth = np.median([wkp[1] for wkp in wkps.values()])
        vector = wkps['UPPER_LIP'] - wkps['TAIL_NOTCH']
        depths.append(depth)
        lengths.append(np.linalg.norm(vector))
    df['depth'] = depths
    df['length'] = lengths
    
    queryCache[pen_id] = { df_start_date: { df_end_date: df } }


In [ ]:
plt.figure(figsize=(20, 10))

mask2 = (df.hour >= 5) & (df.hour <= 15)

avg_weight, raw_weight = get_weight_for_fov(55, 10, pen5)

# counts, bins, _ = plt.hist(pen5.weight * 1000 / 0.83, bins = 50, density = True)
counts, bins, _ = plt.hist(raw_weight * 1000 / 0.83, bins = 50, density = True)
plt.hist(df.estimated_weight_g[mask2], bins = bins, density = True)

In [ ]:
def get_weight_for_fov(degrees, density, df):
    fov = degrees * np.pi / 180
    params_depth = 2
    camera_location = 5
    total_length = 10

    all_weights = []

    num_samples = int(total_length * density)
    
    count = 0
    
    while(len(all_weights) < 3000 and count < 10000):
        count = count + 1
        if count % 1000 == 0:
            print(count)

        results = df.sample(n = num_samples, replace = True)

        x = []

        for index, row in results.iterrows():
            location = np.random.uniform(0, total_length)
            depth = np.random.uniform(0, params_depth)

            x.append([location, row.length, depth, row.weight])
            #x.append([location, row.length, depth, row.estimated_weight_g])

        a = np.array(x)
        b = a[np.argsort(a[:, 2])]

        all_segments = []
        curr_segments = []
        curr_depth = 0

        for row in b:
            curr_depth = row[2]

            band = np.tan(fov / 2) * curr_depth

            lower_bound = camera_location - band
            upper_bound = camera_location + band
            
            if not ((row[0] > lower_bound) and (row[0] + row[1] < upper_bound)):
                if (row[0] > lower_bound) and (row[0] < upper_bound):
                    all_segments.append(row)
                elif ((row[0] + row[1]) > lower_bound) and ((row[0] + row[1]) < upper_bound):
                    all_segments.append(row)
                continue

            is_occluded = False

            for seg in all_segments:
                lower_adj_segment = camera_location + (row[0] - camera_location) * curr_depth / seg[2]
                upper_adj_segment = camera_location + ((row[0] + row[1]) - camera_location) * curr_depth / seg[2]

                if not ((row[0] + row[1]) < lower_adj_segment or row[0] > upper_adj_segment):
                    is_occluded = True

            if not is_occluded:
                all_weights.append(row[3])
                
            all_segments.append(row)

    return np.mean(all_weights), all_weights


In [ ]:
fovs = []
weights = []
raw_weights = []

for degree in np.arange(10, 180, 10):
    avg_weight, raw_weight = get_weight_for_fov(degree)
    fovs.append(degree)
    weights.append(avg_weight)
    raw_weights.append(raw_weight)
    print(degree, avg_weight, len(raw_weight))

In [ ]:
fovs = []
weights = []
raw_weights = []

for density in np.arange(0.25, 5, 0.25):
    avg_weight, raw_weight = get_weight_for_fov(55, density)
    fovs.append(density)
    weights.append(avg_weight)
    raw_weights.append(raw_weight)
    print(density, avg_weight, len(raw_weight))

In [ ]:
for index, density in enumerate(np.arange(8, 20, 2)):
    print(density, weights[index], len(raw_weights[index]))

In [ ]:
plt.figure(figsize=(20, 10))
counts, bins, _ = plt.hist(df.estimated_weight_g, density = True, alpha = 0.5, color = 'red', bins = 20)
counts2, bins, _ = plt.hist(raw_weights[3], density = True, alpha = 0.5, color = 'blue', bins = bins)

(np.array(counts2) / np.sum(counts)) / (np.array(counts) / np.sum(counts2))

In [ ]:
import scipy.stats as stats

mean, std = stats.norm.fit(df.estimated_weight_g)
mean2, std2 = stats.norm.fit(raw_weights[3])

print(mean, std)

# stats.probplot(raw_weights[3], plot = plt)

In [ ]:
stats.probplot(df.estimated_weight_g, plot = plt)

In [ ]:
for i in range(4):
    mean, std = stats.norm.fit(raw_weights[i])
    print(mean, std)

In [ ]:
5632.671352371154 / 5483.3882774105505

In [ ]:
1098.5482565591922 / 1025.0008784912347

In [ ]:
plt.hist(raw_weights[1])

In [ ]:
plt.hist(raw_weights[2])

In [ ]:
plt.hist(raw_weights[3])

In [ ]:
plt.hist(raw_weights[4])

In [ ]:
plt.plot(np.arange(1, 5, 1), weights / np.mean(df.estimated_weight_g))

In [ ]:
5500 / 5700

In [ ]:
np.mean(all_weights), np.mean(df.estimated_weight_g)

In [ ]:
np.mean(all_weights) / np.mean(df.estimated_weight_g)

In [ ]:
plt.plot(fovs, weights)

In [ ]:
plt.plot(fovs, np.array(weights) / np.mean(df.estimated_weight_g))

In [ ]:
weights2 = weights